In [1]:
#put the these lines before importing any module from keras.
import tensorflow as tf

from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.95
config.gpu_options.visible_device_list = "0" #only the gpu 0 is allowed

sess = tf.Session(config=config)
graph = tf.get_default_graph()
set_session(sess)

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

/home/zdai/anaconda3/envs/DLpy37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zdai/anaconda3/envs/DLpy37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zdai/anaconda3/envs/DLpy37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zdai/anaconda3/envs/DLpy37/lib/python3.7/site-

['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
import os
from os.path import join
import numpy as np
import keras
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib auto
import keras
from keras.models import Model
from keras.layers import Input, add, LSTM, RepeatVector, TimeDistributed, concatenate, Conv2DTranspose, Add
from keras.layers.core import Layer, Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.utils import np_utils, plot_model
from keras_segmentation.models.model_utils import get_segmentation_model

Using matplotlib backend: Qt5Agg


In [21]:
vidcap = cv2.VideoCapture(join('dataset2', 'SampleASAPSData.mp4'))
print(cv2.__version__)

4.5.2


In [22]:
fps = vidcap.get(cv2.CAP_PROP_FPS)      # OpenCV2 version 2 used "CV_CAP_PROP_FPS"
frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count/fps

print('fps = ' + str(fps))
print('number of frames = ' + str(frame_count))
print('duration (S) = ' + str(duration))
minutes = int(duration/60)
seconds = duration%60
print('duration (M:S) = ' + str(minutes) + ':' + str(seconds))

fps = 30.0
number of frames = 30211
duration (S) = 1007.0333333333333
duration (M:S) = 16:47.0333333333333


In [23]:
count = 0

while vidcap.isOpened():
    success, image = vidcap.read()
    if success:
        
        #cv2.imshow('Images', image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if count >= 11200 and count < 11700:
            cv2.imwrite("dataset2/images/frame%d.png" % count, image)
        elif count >= 12000:
            break
        
        count += 1
    else:
        print("Read Image from Video Failed!")
        break
        
vidcap.release()
cv2.destroyAllWindows()

In [14]:
data_dir = join('dataset2', 'images')

height, width = 224, 224

image0 = cv2.imread(join(data_dir, 'frame11386.png'))
img0 = cv2.resize(image0, (width, height))

fig = plt.figure(figsize=(5,5))

ax1 = fig.add_subplot(2,1,1)
ax1.imshow(image0)
ax2 = fig.add_subplot(2,1,2)
ax2.imshow(img0)

plt.show()

In [4]:
# Load Model
from keras_segmentation.models.unet import unet_mini, vgg_unet

model0 = unet_mini(n_classes=12 ,  input_height=height, input_width=width )
model0.summary()

model0.load_weights(join('checkpoints', 'mini_unet0'))


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 224, 224, 32) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 224, 224, 32) 9248        dropout_1[0][0]                  
___________________________________________________________________________________________

In [8]:
def give_color_to_seg_img(seg, n_classes):
    '''
    seg : (input_width,input_height,3)
    '''
    
    if len(seg.shape)==3:
        seg = seg[:,:,0]
    seg_img = np.zeros( (seg.shape[0],seg.shape[1],3) ).astype('float')
    colors = sns.color_palette("hls", n_classes)
    
    for c in range(n_classes):
        segc = (seg == c)
        seg_img[:,:,0] += (segc*( colors[c][0] ))
        seg_img[:,:,1] += (segc*( colors[c][1] ))
        seg_img[:,:,2] += (segc*( colors[c][2] ))

    return(seg_img)

def getImageArr( path , width , height ):
        img = cv2.imread(path, 1)
        img = np.float32(cv2.resize(img, ( width , height ))) / 127.5 - 1
        return img

def getSegmentationArr( path , nClasses ,  width , height  ):

    seg_labels = np.zeros((  height , width  , nClasses ))
    img = cv2.imread(path, 1)
    img = cv2.resize(img, ( width , height ))
    img = img[:, : , 0]

    for c in range(nClasses):
        seg_labels[: , : , c ] = (img == c ).astype(int)
    ##seg_labels = np.reshape(seg_labels, ( width*height,nClasses  ))
    return seg_labels

In [11]:
x_test0 = img0.reshape(1, height, width, 3)
y_pred = model0.predict(x_test0).reshape(-1, height, width, 12)
y_predi = np.argmax(y_pred, axis=3)

print(y_predi.shape)
y_pred.shape

(1, 224, 224)


(1, 224, 224, 12)

In [13]:
y_pred0 = y_pred.reshape(height, width, 12)

plt.imshow(give_color_to_seg_img(y_pred0, 12))
plt.show()